In [2]:
import pandas as pd

x_df = pd.read_csv(r"C:\Users\91790\Downloads\dataset and all\training_set_features.csv")
test_df = pd.read_csv(r"C:\Users\91790\Downloads\dataset and all\test_set_features.csv")
x_df = x_df.drop(columns="respondent_id")
test_id = test_df["respondent_id"]
test_df = test_df.drop(columns="respondent_id")

In [3]:
x_df.head()

,xyz_concern,xyz_knowledge,behavioral_antiviral_meds,behavioral_avoidance,behavioral_face_mask,behavioral_wash_hands,behavioral_large_gatherings,behavioral_outside_home,behavioral_touch_face,doctor_recc_xyz,...,income_poverty,marital_status,rent_or_own,employment_status,hhs_geo_region,census_msa,household_adults,household_children,employment_industry,employment_occupation
0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,...,Below Poverty,Not Married,Own,Not in Labor Force,oxchjgsf,Non-MSA,0.0,0.0,NaN,NaN
1,3.0,2.0,0.0,1.0,0.0,1.0,0.0,1.0,1.0,0.0,...,Below Poverty,Not Married,Rent,Employed,bhuqouqj,"MSA, Not Principle City",0.0,0.0,pxcmvdjn,xgwztkwe
2,1.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,NaN,...,"<= $75,000, Above Poverty",Not Married,Own,Employed,qufhixun,"MSA, Not Principle City",2.0,0.0,rucpziij,xtkaffoo
3,1.0,1.0,0.0,1.0,0.0,1.0,1.0,0.0,0.0,0.0,...,Below Poverty,Not Married,Rent,Not in Labor Force,lrircsnp,"MSA, Principle City",0.0,0.0,NaN,NaN
4,2.0,1.0,0.0,1.0,0.0,1.0,1.0,0.0,1.0,0.0,...,"<= $75,000, Above Poverty",Married,Own,Employed,qufhixun,"MSA, Not Principle City",1.0,0.0,wxleyezf,emcorrxb


In [5]:

def na_mode(df):
    for col in df.columns:
        df[col].fillna(df[col].mode()[0], inplace=True)

In [6]:

def object_num(df, col):
    mp = {}
    lst = df[col].unique()
    for idx, val in enumerate(lst):
        mp[val] = idx
    df[col] = df[col].map(mp)

In [12]:
def col_num(df):
    col_to_change = []
    for col in df.columns:
        if(df[col].dtype == "object"):
            col_to_change.append(col)

    for col in col_to_change:
        object_num(df, col)

In [13]:
na_mode(x_df)
col_num(x_df)

C:\Users\91790\AppData\Local\Temp\ipykernel_23484\3894437392.py:3: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df[col].fillna(df[col].mode()[0], inplace=True)


In [14]:
x_df.shape


(26707, 35)

In [15]:

# Importing the libraries

import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.metrics import AUC

In [17]:

def build_model(input_shape):
    model = Sequential()
    model.add(Dense(128, input_shape=input_shape, activation='relu'))
    model.add(Dropout(0.3))
    model.add(Dense(64, activation='relu'))
    model.add(Dropout(0.3))
    model.add(Dense(32, activation='relu'))
    model.add(Dense(2, activation='sigmoid'))
    return model

In [18]:
input_shape = (x_df.shape[1],) # Number of columns (features) in the training set
model = build_model(input_shape)
 
# Compile the model
model.compile(optimizer=Adam(learning_rate=0.001), loss='binary_crossentropy', metrics=['accuracy', AUC(name='auc')])

# Summary of the model
model.summary()


Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               (None, 128)               4608      
                                                                 
 dropout (Dropout)           (None, 128)               0         
                                                                 
 dense_1 (Dense)             (None, 64)                8256      
                                                                 
 dropout_1 (Dropout)         (None, 64)                0         
                                                                 
 dense_2 (Dense)             (None, 32)                2080      
                                                                 
 dense_3 (Dense)             (None, 2)                 66        
                                                                 
Total params: 15010 (58.63 KB)
Trainable params: 15010 

In [19]:
y_df = pd.read_csv(r"C:\Users\91790\Downloads\dataset and all\training_set_labels.csv")
y_df.head()

,respondent_id,xyz_vaccine,seasonal_vaccine
0,0,0,0
1,1,0,1
2,2,0,0
3,3,0,1
4,4,0,0


In [20]:
y_df = y_df.drop(columns="respondent_id") # Drop the respondent_id column since its not required during training
y_df.shape

(26707, 2)

In [21]:
history = model.fit(x_df, y_df, epochs=500, batch_size=32, validation_split=0.2, verbose=1)


Epoch 1/500


668/668 [==============================] - 5s 4ms/step - loss: 0.5405 - accuracy: 0.2971 - auc: 0.7627 - val_loss: 0.4915 - val_accuracy: 0.2896 - val_auc: 0.8218
Epoch 2/500
668/668 [==============================] - 3s 4ms/step - loss: 0.4892 - accuracy: 0.2923 - auc: 0.8149 - val_loss: 0.4709 - val_accuracy: 0.2898 - val_auc: 0.8297
Epoch 3/500
668/668 [==============================] - 3s 4ms/step - loss: 0.4747 - accuracy: 0.2920 - auc: 0.8283 - val_loss: 0.4656 - val_accuracy: 0.2907 - val_auc: 0.8349
Epoch 4/500
668/668 [==============================] - 3s 4ms/step - loss: 0.4673 - accuracy: 0.3000 - auc: 0.8343 - val_loss: 0.4582 - val_accuracy: 0.2943 - val_auc: 0.8411
Epoch 5/500
668/668 [==============================] - 3s 4ms/step - loss: 0.4617 - accuracy: 0.2993 - auc: 0.8390 - val_loss: 0.4563 - val_accuracy: 0.2986 - val_auc: 0.8414
Epoch 6/500
668/668 [==============================] - 2s 4ms/step - loss: 0.4590 - accuracy: 0.3088 - auc: 0.8414 - val_lo

In [41]:
na_mode(test_df)
col_num(test_df)

C:\Users\91790\AppData\Local\Temp\ipykernel_23484\3894437392.py:3: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df[col].fillna(df[col].mode()[0], inplace=True)
C:\Users\91790\AppData\Local\Temp\ipykernel_23484\3894437392.py:3: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For exam

In [42]:
test_df.shape


(26708, 35)

In [43]:

y_pred_probs = model.predict(test_df)
y_pred_probs

835/835 [==============================] - 2s 2ms/step


array([[0.14737377, 0.38649815],
       [0.10170209, 0.10697957],
       [0.2516517 , 0.5155811 ],
       ...,
       [0.13342808, 0.30096748],
       [0.04885807, 0.3098518 ],
       [0.26772603, 0.46614876]], dtype=float32)

In [44]:
y_pred_probs.shape

(26708, 2)

In [45]:
y_pred = pd.DataFrame(y_pred_probs, columns=['h1n1_vaccine', 'seasonal_vaccine'])


In [46]:
y_pred.head()


,h1n1_vaccine,seasonal_vaccine
0,0.147374,0.386498
1,0.101702,0.106980
2,0.251652,0.515581
3,0.353156,0.580112
4,0.555088,0.772107


In [47]:
test_id = pd.DataFrame(test_id, columns=['respondent_id'])


In [48]:
test_id.head()


,respondent_id
0,26707
1,26708
2,26709
3,26710
4,26711


In [49]:
combined = pd.concat([test_id, y_pred], axis=1)


In [50]:
combined.shape


(26708, 3)

In [51]:
combined.to_csv('submission.csv', index=False)

In [52]:
from sklearn.metrics import roc_auc_score
results = model.evaluate(x_df, y_df, verbose=1)

y_pred_proba = model.predict(x_df)

roc_auc_xyz = roc_auc_score(y_df.iloc[:, 0], y_pred_proba[:, 0])
roc_auc_seasonal = roc_auc_score(y_df.iloc[:, 1], y_pred_proba[:, 1])

print(f"ROC AUC for XYZ Vaccine: {roc_auc_xyz}")
print(f"ROC AUC for Seasonal Vaccine: {roc_auc_seasonal}")


835/835 [==============================] - 2s 2ms/step
ROC AUC for XYZ Vaccine: 0.9049342766183044
ROC AUC for Seasonal Vaccine: 0.908805198467006
